## Binary Class SVM

### One vs All

In [87]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import csv,sys
import math,random
import subprocess
from sklearn.metrics import roc_curve
from sklearn.svm import OneClassSVM
from sklearn.svm import SVC

In [15]:
df = pd.read_csv("DSL-StrongPasswordData.csv")
s002_data = df.subject.apply(lambda i: 1 if i=='s002' else 0 )
print(sum(s002_data))
df['result'] = s002_data
df.head()
df = df.drop(['subject','sessionIndex','rep'],axis = 1)
X = df.drop(['result'],axis = 1)
y = df.result
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

400


In [17]:
#model = SVC(kernel='rbf') 0.98088
#model = SVC(kernel='poly') 0.97990
#model = SVC(kernel='linear') 0.9786
model = SVC(kernel='sigmoid') 0.9612
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9612745098039216

### One vs Random of same size

In [18]:
data = pd.read_csv("DSL-StrongPasswordData.csv")
data.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,1.1885,1.0468,0.1146,1.6055,1.4909,0.1067,0.7590,0.6523,0.1016,0.2136,0.1120,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,1.1970,1.1141,0.0689,0.7822,0.7133,0.1570,0.7877,0.6307,0.1066,0.1684,0.0618,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,1.0408,0.9600,0.0892,0.6203,0.5311,0.1454,0.7195,0.5741,0.1365,0.2931,0.1566,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,1.0556,0.9656,0.0913,1.2564,1.1651,0.1454,0.7550,0.6096,0.0956,0.1530,0.0574,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,0.8629,0.7824,0.0742,0.8955,0.8213,0.1243,0.7632,0.6389,0.0430,0.1975,0.1545,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [54]:
pos_s2data = data[data['subject']=='s002']
pos_s2data['output'] = pos_s2data.apply(lambda i: 1 if i['subject']=='s002' else 0 , axis = 1) 

neg_s2data = data[data['subject']!='s002']
neg_s2data['output'] = neg_s2data.apply(lambda i: 1 if i['subject']=='s002' else 0 , axis = 1) 
neg_s2data = neg_s2data.sample(400)

s2data = pd.concat([pos_s2data, neg_s2data],ignore_index=True)
s2data = s2data.sample(frac=1)
s2data

X = s2data.drop(['subject','sessionIndex','rep','output'],axis = 1)
y = s2data['output']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [55]:
#model = SVC(kernel='rbf') 0.8625
#model = SVC(kernel='poly') 0.8533
#model = SVC(kernel='linear') 0.826
#model = SVC(kernel='sigmoid') Bad Performance
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.83125

### Considering Test data of 100 from 400 samples of each user

In [47]:
pos_s2data = data[data['subject']=='s002']
pos_s2data['output'] = pos_s2data.apply(lambda i: 1 if i['subject']=='s002' else 0 , axis = 1) 
pos_s2data = pos_s2data.sample(frac=1)
pos_s2test_data = pos_s2data[:101]
pos_s2data = pos_s2data[101:]

neg_s2data = data[data['subject']!='s002']
neg_s2data['output'] = neg_s2data.apply(lambda i: 1 if i['subject']=='s002' else 0 , axis = 1) 
neg_s2data = neg_s2data.sample(400)
neg_s2test_data = neg_s2data[:101]
neg_s2data = neg_s2data[101:]

s2data = pd.concat([pos_s2data, neg_s2data],ignore_index=True)
s2data = s2data.sample(frac=1)
s2data

X_train = s2data.drop(['subject','sessionIndex','rep','output'],axis = 1)
y_train = s2data['output']

s2data = pd.concat([pos_s2test_data, neg_s2test_data],ignore_index=True)
s2data = s2data.sample(frac=1)
s2data

X_test = s2data.drop(['subject','sessionIndex','rep','output'],axis = 1)
y_test = s2data['output']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
#model = SVC(kernel='rbf') 0.8811
#model = SVC(kernel='poly') 0.8712
#model = SVC(kernel='linear') 0.811
#model = SVC(kernel='sigmoid') Bad Performance

model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8625

## Multiclass SVM

### Splitting 300-100

In [57]:
df2 = pd.read_csv("DSL-StrongPasswordData.csv")
df2.head()
print(int(df2.iloc[2].subject[2:4]))
converted_col = df2.subject.apply(lambda i: int(i[2:4]) )
converted_col.head()

2


0    2
1    2
2    2
3    2
4    2
Name: subject, dtype: int64

In [58]:
df2 = df2.drop(['subject','sessionIndex','rep'],axis = 1)
df2['subject'] = converted_col
df2.head()

,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return,subject
0,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,1.1885,1.0468,0.1146,1.6055,1.4909,0.1067,0.7590,0.6523,0.1016,0.2136,0.1120,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742,2
1,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,1.1970,1.1141,0.0689,0.7822,0.7133,0.1570,0.7877,0.6307,0.1066,0.1684,0.0618,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747,2
2,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,1.0408,0.9600,0.0892,0.6203,0.5311,0.1454,0.7195,0.5741,0.1365,0.2931,0.1566,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945,2
3,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,1.0556,0.9656,0.0913,1.2564,1.1651,0.1454,0.7550,0.6096,0.0956,0.1530,0.0574,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813,2
4,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,0.8629,0.7824,0.0742,0.8955,0.8213,0.1243,0.7632,0.6389,0.0430,0.1975,0.1545,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818,2


In [84]:
Test_data = pd.DataFrame()
Train_data = pd.DataFrame()
start = 0
end = 350
for i in range (51):
    Train_data = Train_data.append(df2[start:end])
    Test_data = Test_data.append(df2[end:end+50])
    start = end+50
    end = start+350
print(len(Train_data))
print(len(Test_data))   
Train_data = Train_data.sample(frac = 1)
Test_data = Test_data.sample(frac = 1)

17850
2550


In [85]:
X_train = Train_data.drop(['subject'],axis = 1)
y_train = Train_data['subject']
X_test = Test_data.drop(['subject'],axis = 1)
y_test = Test_data['subject']

In [86]:
#model = SVC(kernel='rbf') 0.7870
#model = SVC(kernel='linear') 0.7431

model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7870588235294118

## One Class SVM

In [88]:
data = pd.read_csv("DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()

In [89]:
def evaluateEER(user_scores, imposter_scores):
    #print user_scores
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer

In [96]:
def evaluate():
    eers = []
    for subject in subjects:
            
        user_scores = []
        imposter_scores = []
    
        # Consider current subject as genuine and rest as imposters
        genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]
    
        # genuine user's first 200 time vectors for training
        train = genuine_user_data[:200]
        train = train.values
        train = np.array(train)
        #labels = np.ones(len(train),dtype = np.float64)
        #print type(labels[0])
        test_genuine = genuine_user_data[200:]
        test_genuine = test_genuine.values
        test_genuine = np.array(test_genuine)
        test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
        test_imposter = test_imposter.values
        test_imposter = np.array(test_imposter)
        clf = OneClassSVM(kernel='poly',gamma=26)
        clf.fit(train)
        #print len(test_genuine)
        #print len(test_imposter)
        user_scores = -clf.decision_function(test_genuine)
        imposter_scores = -clf.decision_function(test_imposter)
        user_scores = list(user_scores)
        imposter_scores = list(imposter_scores)
        eers.append(evaluateEER(user_scores, imposter_scores))
    #print eers   
    return np.mean(eers), np.std(eers)

In [97]:
evaluate() 
# For RBF Kernel (0.12054244703221502, 0.058709278630758664)
# For Linear Kernel (0.8161469845638643, 0.1510965776375408)
# For Poly Kernel (0.8166892639378132, 0.14922221119496795)

#Clearly RBF Kernel is best suited for all the cases

(0.8166892639378132, 0.14922221119496795)